# New dataset for abundance analysis

## Things to do
* Separate samples into insulin/basal
* For abundance cut-offs, averages the metabolite abundance across samples in the same group
* For DE list, use paired t-tests to determine DE metabolites. Beware of the zeros.
* Check how long the list is

### Next step: paired t-tests

## class definition of a sample

In [1]:
class MetabolomicsSample:
    def __init__(self, sample_id, liver_fat, infusion, serum, patient_id):
        '''
        id, liver fat, insulin infusion, serum type, patient, compound:abundance
        '''
        self.sample_id = sample_id
        self.liver_fat = liver_fat
        self.infusion = infusion
        self.serum = serum
        self.patient_id = patient_id
        self._data = dict()
    
    def __repr__(self):
        return self.sample_id
    
    def getSampleInfo(self):
        print(self.sample_id)
        print("Liver fat(percentage):", self.liver_fat)
        print("Insulin infusion: " + self.infusion)
        print("Serum type: " + self.serum)
        print("Patient id: " + self.patient_id)
    
    def addOmicsData(self, metabolite_id, value):
        '''
        Find the existing entry of the metabolite in the data dictionary, add value to the existing entry
        If no existing entry can be found, create an entry for the metabolite
        Values have to be in float!!
        '''
        if not isinstance(value, float):
            print("Data values must be floats")
            return 0
        self._data[metabolite_id] = self._data.get(metabolite_id, 0) + value
        
    def getOmicsData(self):
        return self._data
    
    def clearOmicsData(self):
        self._data = dict()
        
    def avgOmicsData(self, replicate):
        '''
        Average the omics data from two replicates
        Return with a new object that has the average omics data
        ***If one replicate has 0 for a metabolite, the average will be 0
        
        Could potentially add an attribute that states whether a study has been averaged or not
        '''
        if not isinstance(replicate, MetabolomicsSample):
            print("Replicate must be a MetabolomicsSample sample")
            return 0
        elif (replicate.liver_fat != self.liver_fat or 
              replicate.infusion != self.infusion or 
              replicate.serum != self.serum or 
              replicate.patient_id != self.patient_id):
            print("Study info do not match")
            return 0
        mean_study = MetabolomicsSample(self.sample_id, self.liver_fat, 
                                        self.infusion, self.serum, self.patient_id)
        for metabolite in self.getOmicsData():
            if self.getOmicsData()[metabolite] == 0 or replicate.getOmicsData()[metabolite] == 0:
                avg_abundance = 0.0
            else:
                avg_abundance = (self.getOmicsData()[metabolite] + replicate.getOmicsData()[metabolite]) / 2
            mean_study.addOmicsData(metabolite, avg_abundance)
        return mean_study

## MTBLS298
242 Unique Chebi IDs

In [2]:
directory = '/data/zx2313/MTBLS298/' # The location of the maf file
maf = directory + 'm_catheterization_study_metabolite_profiling_mass_spectrometry_v2_maf.tsv'
sample_info = directory + 's_Catheterization study.txt'

In [3]:
# Initialise metabolomics samples
met_studies = []
with open(sample_info, 'r') as fh:
    for line in fh.readlines()[1:]:
        fields = line.rstrip().split('\t')
        sample_id = fields[8][1:-1]
        liver_fat = int(fields[9][1:-1])
        infusion = fields[16][1:-1]
        serum = fields[19][1:-1]
        patient_id = fields[22][1:-1]
        met_studies.append(MetabolomicsSample(sample_id, liver_fat, infusion, serum, patient_id))

In [ ]:
for i in range(0, len(met_studies)):
    print(len(met_studies[i].getOmicsData()))

In [11]:
print(met_studies)

[112Vein_pl01_GGT3_rep1run1_291009_1, 112Vein_pl01_GGT3_rep1run2_291009_1, 112Vein_pl05_GGT3_rep1run1_291009_1, 112Vein_pl05_GGT3_rep1run2_291009_1, 112Vein_pl09_GGT3_rep1run1_291009_1, 112Vein_pl09_GGT3_rep1run2_291009_1, 112Vein_pl13_GGT3_rep1run1_291009_1, 112Vein_pl13_GGT3_rep1run2_291009_1, 112Vein_pl17_GGT3_rep1run1_291009_1, 112Vein_pl17_GGT3_rep1run2_291009_1, 112Vein_pl21_GGT3_rep1run1_291009_1, 112Vein_pl21_GGT3_rep1run2_291009_1, 112Vein_pl25_GGT3_rep1run1_291009_1, 112Vein_pl25_GGT3_rep1run2_291009_1, 112Vein_pl29_GGT3_rep1run1_291009_1, 112Vein_pl29_GGT3_rep1run2_291009_1, 112Vein_pl33_GGT3_rep1run1_291009_1, 112Vein_pl33_GGT3_rep1run2_291009_1, 112Vein_pl02_GGT3_rep1run1_291009_1, 112Vein_pl02_GGT3_rep1run2_291009_1, 112Vein_pl06_GGT3_rep1run1_291009_1, 112Vein_pl06_GGT3_rep1run2_291009_1, 112Vein_pl26_GGT3_rep1run1_291009_1, 112Vein_pl26_GGT3_rep1run2_291009_1, 112Vein_pl30_GGT3_rep1run1_291009_1, 112Vein_pl30_GGT3_rep1run2_291009_1, 112Vein_pl34_GGT3_rep1run1_291009_1, 

In [10]:
with open(maf, 'r') as fh:
    study_indices = []
    lines = fh.readlines()
    for line in lines[0:1]:
        fields = line.rstrip().split('\t')
        #print(fields[0], fields[4], fields[11], fields[14], fields[16], fields[17], fields[21:81])# 0 4 11 14 16 17 21-80
        for field in fields[21:81]:
            sample_id = field[1:-5]
            for index in range(0, len(met_studies)):
                study = met_studies[index]
                if sample_id == study.sample_id:
                    study_indices.append(index)
                    
    for line in lines[1:]:
        fields = line.rstrip().split('\t')
        database_id = fields[0][1:-1]
        if not database_id.startswith('CHEBI'):
            continue # If the metabolite is not chebi, go to the next line
        else:
            converted_id = conv_chebi_kegg(database_id, ch)
        if not (converted_id.startswith('C') and len(converted_id) == 6):
            continue # If the converted id is not kegg, go to the next line
        for index in range(21, 81): #fields[21:81]:
            study_index = study_indices[index - 21]
            omics_value = float(fields[index][1:-1])
            met_studies[study_index].addOmicsData(converted_id, omics_value)

In [12]:
met_studies.sort(key = lambda x: x.patient_id)

In [53]:
basal_studies = []
insulin_studies = []
for study in met_studies:
    if study.infusion == 'Basal':
        basal_studies.append(study)
    elif study.infusion == 'Insulin':
        insulin_studies.append(study)

In [54]:
for study in basal_studies:
    study.getSampleInfo()

112Vein_pl01_GGT3_rep1run1_291009_1
Liver fat(percentage): 65
Insulin infusion: Basal
Serum type: Artery
Patient id: 1
112Vein_pl01_GGT3_rep1run2_291009_1
Liver fat(percentage): 65
Insulin infusion: Basal
Serum type: Artery
Patient id: 1
112Vein_pl02_GGT3_rep1run1_291009_1
Liver fat(percentage): 65
Insulin infusion: Basal
Serum type: Vein
Patient id: 1
112Vein_pl02_GGT3_rep1run2_291009_1
Liver fat(percentage): 65
Insulin infusion: Basal
Serum type: Vein
Patient id: 1
112Vein_pl05_GGT3_rep1run1_291009_1
Liver fat(percentage): 70
Insulin infusion: Basal
Serum type: Artery
Patient id: 2
112Vein_pl05_GGT3_rep1run2_291009_1
Liver fat(percentage): 70
Insulin infusion: Basal
Serum type: Artery
Patient id: 2
112Vein_pl06_GGT3_rep1run1_291009_1
Liver fat(percentage): 70
Insulin infusion: Basal
Serum type: Vein
Patient id: 2
112Vein_pl06_GGT3_rep1run2_291009_1
Liver fat(percentage): 70
Insulin infusion: Basal
Serum type: Vein
Patient id: 2
112Vein_pl09_GGT3_rep1run1_291009_1
Liver fat(percentage

In [55]:
for study in insulin_studies:
    study.getSampleInfo()

112Vein_pl03_GGT3_rep1run1_291009_1
Liver fat(percentage): 65
Insulin infusion: Insulin
Serum type: Artery
Patient id: 1
112Vein_pl03_GGT3_rep1run2_291009_1
Liver fat(percentage): 65
Insulin infusion: Insulin
Serum type: Artery
Patient id: 1
112Vein_pl04_GGT3_rep1run1_291009_1
Liver fat(percentage): 65
Insulin infusion: Insulin
Serum type: Vein
Patient id: 1
112Vein_pl04_GGT3_rep1run2_291009_1
Liver fat(percentage): 65
Insulin infusion: Insulin
Serum type: Vein
Patient id: 1
112Vein_pl07_GGT3_rep1run1_291009_1
Liver fat(percentage): 70
Insulin infusion: Insulin
Serum type: Artery
Patient id: 2
112Vein_pl07_GGT3_rep1run2_291009_1
Liver fat(percentage): 70
Insulin infusion: Insulin
Serum type: Artery
Patient id: 2
112Vein_pl08_GGT3_rep1run1_291009_1
Liver fat(percentage): 70
Insulin infusion: Insulin
Serum type: Vein
Patient id: 2
112Vein_pl08_GGT3_rep1run2_291009_1
Liver fat(percentage): 70
Insulin infusion: Insulin
Serum type: Vein
Patient id: 2
112Vein_pl11_GGT3_rep1run1_291009_1
Live

In [56]:
avg_basal_studies = avg_replicates_study(basal_studies)
avg_insulin_studies = avg_replicates_study(insulin_studies)

In [70]:
for study in avg_basal_studies:
    print(study.patient_id)

1
1
2
2
3
4
5
6
8
8
9
9


In [71]:
for study in avg_insulin_studies:
    print(study.patient_id)

1
1
2
2
3
4
5
6
8
8
9
9


In [65]:
avg_basal_studies = avg_basal_studies[:8] + avg_basal_studies[10:]

In [66]:
avg_insulin_studies = avg_insulin_studies[:5] + avg_insulin_studies[6:7] + avg_insulin_studies[8:9] + avg_insulin_studies[10:11] + avg_insulin_studies[12:]

## Paired t-tests
- Conditions: basal vs insulin
- samples from the same patient and serum were paired together (12 vs 12)
- If a zero abundance is present for one metabolite, its abundance in the related sample will be also removed and excluded from the paired t-tests
- CONCERN: Small sample size (< 8) for Wilcoxon tests??
- before multiple-testing corrections: 11 DE metabolites
- after multiple-testing corrections: 1 DE metabolite

In [80]:
import scipy.stats
# FDR
from rpy2.robjects.packages import importr
from rpy2.robjects.vectors import FloatVector

In [81]:
# go through one metabolite in all samples
# do the same thing with the other condition
# if the paired t-test is significant, add the metabolite to the DE list
# if any zero is present in one sample, remove the related data in the other condition
p_cutoff = 0.05
p_vals = []
tested_metabolites = []
all_metabolites = list(avg_basal_studies[0].getOmicsData().keys())
for metabolite in all_metabolites:
    b_study_paired_abun = []
    i_study_paired_abun = []
    for index in range(0, len(avg_basal_studies)):
        b_study = avg_basal_studies[index]
        i_study = avg_insulin_studies[index]
        if b_study.getOmicsData()[metabolite] != 0.0 and i_study.getOmicsData()[metabolite] != 0.0:
            # if neither of the abundance is zero, add the data points to the paired list
            b_study_paired_abun.append(b_study.getOmicsData()[metabolite])
            i_study_paired_abun.append(i_study.getOmicsData()[metabolite])
    #print(len(b_study_paired_abun), len(i_study_paired_abun))
    p_val = scipy.stats.ttest_rel(b_study_paired_abun, i_study_paired_abun)[1]
    if p_val > 0: # remove nan in the results
        p_vals.append(p_val)
        tested_metabolites.append(metabolite)
p_vals = list(importr('stats').p_adjust(FloatVector(p_vals), method = 'BH'))

/project/soft/linux64/anaconda/Anaconda3-2.4.1-Linux-x86_64/lib/python3.5/site-packages/numpy/core/_methods.py:82: RuntimeWarning: Degrees of freedom <= 0 for slice
  warnings.warn("Degrees of freedom <= 0 for slice", RuntimeWarning)


In [83]:
for k in p_vals:
    if k < 0.05:
        print(k)

0.006929651989095781


In [68]:
de_metabolites

[C00233,
 C00022,
 C16439,
 C02287,
 C00219,
 C01733,
 C00302,
 C00042,
 C00164,
 C08261,
 C01530]

## Conversion: Chebi to KEGG

In [39]:
from bioservices import KEGG, ChEBI
import ora_msc

In [40]:
ch = ChEBI()

In [41]:
kegg = KEGG()
kegg.organism = 'hsa'

In [42]:
hsa_pathways = kegg.pathwayIds
pathway_2_compounds = dict()
for pathway in hsa_pathways:
    parsed_output = kegg.parse(kegg.get(pathway)) # parsed_ouput has lots of information about the pathway
    try:
        compounds = set(parsed_output['COMPOUND'].keys())
        pathway_2_compounds[pathway] = compounds
    except KeyError: # Some pathways do not have defined compounds
        #name = parsed_output['NAME']
        #print(pathway, name)
        pass

In [72]:
for pathway in hsa_pathways:
    result = ora_msc.ora(set(de_metabolites), pathway, set(avg_basal_studies[0].getOmicsData().keys()), pathway_2_compounds)
    if len(result) == 3:
        print(result)

(0.35346546385164174, 'path:hsa00010', 4)
(0.14944764664154689, 'path:hsa00020', 7)
(0.10185185185185021, 'path:hsa00030', 1)
(0.35346546385164174, 'path:hsa00040', 4)
(0.35346546385164174, 'path:hsa00053', 4)
(0.48359321403234107, 'path:hsa00061', 6)
(0.10185185185185021, 'path:hsa00072', 1)
(0.19418483904460709, 'path:hsa00190', 2)
(0.14944764664154689, 'path:hsa00250', 7)
(0.35346546385164174, 'path:hsa00260', 4)
(0.19418483904460709, 'path:hsa00270', 2)
(0.0095188646590510222, 'path:hsa00280', 2)
(0.050832276718370019, 'path:hsa00290', 4)
(0.19418483904460709, 'path:hsa00310', 2)
(0.42184892440580191, 'path:hsa00330', 5)
(0.047549790447018597, 'path:hsa00350', 9)
(0.14944764664154689, 'path:hsa00360', 7)
(0.19418483904460709, 'path:hsa00430', 2)
(0.27780716706829001, 'path:hsa00440', 3)
(0.10185185185185021, 'path:hsa00590', 1)
(0.19418483904460709, 'path:hsa00591', 2)
(0.07993162163504057, 'path:hsa00620', 5)
(0.14944764664154689, 'path:hsa00630', 7)
(0.050832276718370019, 'path:h

In [5]:
def conv_chebi_kegg(chebi_id, chebi_instance):
    '''
    converting chebi ids to kegg ids
    if the chebi entry does not have database links
    return the ascii name of that entry
    '''
    res = chebi_instance.getCompleteEntity(chebi_id)
    try:
        for link in res.DatabaseLinks:
            if link.type == 'KEGG COMPOUND accession':
                kegg_id = link.data
                return kegg_id
    except AttributeError:
        pass
    return res.chebiAsciiName

def avg_replicates_study(repstudies):
    '''
    Only works when there are exactly two replicates
    Must sort the input list by patient id first!!!
    '''
    avg_studies = []
    for i in range(0, len(repstudies), 2):
        study_rep1 = repstudies[i]
        study_rep2 = repstudies[i+1]
        study_avg = study_rep1.avgOmicsData(study_rep2)
        avg_studies.append(study_avg)
    return avg_studies